# 심층 합성곱 생성적 적대 신경망 (Deep Convolutional Generative Adversarial Networks, DCGAN)

## 생성적 적대 신경망(GANs) 

- [생성적 적대 신경망](https://arxiv.org/abs/1406.2661) (Generative Adversarial Networks, GANs)은 요즘 컴퓨터 과학에서 가장 흥미로운 아이디어 중 하나 입니다.
- 두개의 모델이 적대적인 과정을 통해 동시에 훈련 됩니다.
- *생성자* ("예술가")는 진짜처럼 보이는 이미지를 생성하도록 배우는 와중에, *감별자* ("예술비평가")는 가짜의 이미지로부터 진짜를 구별하게 되는 것을 배우게 됩니다.

![생성자와 감별자를 그린 도표](https://tensorflow.org/tutorials/generative/images/gan1.png)



- 훈련과정 동안 *생성자*는 점차 실제같은 이미지를 더 잘 생성하게 됩니다.
- *감별자*는 점차 진짜와 가짜를 더 잘 구별하게 됩니다.
- 이 과정은 *감별자*가 가짜 이미지에서 진짜 이미지를 더이상 구별하지 못하게 될때, 평형상태에 도달 합니다.

![생성자와 감별자를 그린 두번째 도표](https://tensorflow.org/tutorials/generative/images/gan2.png)

- 이 과정을 MNIST 데이터를 이용하여 구현해 봅니다.
- 아래의 애니메이션은 50 에포크(epoch)동안 훈련한 *생성자*가 생성해낸 연속된 이미지들을 보여줍니다.
- 이미지들은 랜덤한 잡음으로 부터 시작되었고, 점차 시간이 지남에 따라 손으로 쓴 숫자들을 닮아가게 됩니다.

![출력 예시](https://tensorflow.org/images/gan/dcgan.gif)

### 데이터셋 로딩 및 준비
- 생성자와 감별자를 훈련하기위해 MNIST 데이터셋을 사용  합니다.
- 생성자는 손글씨 숫자 데이터를 닮은 숫자들을 생성할 것 입니다.

MNIST 손글씨는 (n_samples, 28, 28)의 shape을 가지는 0과 1 사이의 픽셀값을 갖는 이미지 데이터입니다. 이를 (n_samples, 28, 28, 1)의 shape을 가지는 0과 1 사이의 픽셀값을 갖는 이미지 데이터로 변환하고, 각 픽셀값을 -1에서 1 사이의 값으로 normalize해 줍니다.

from_tensor_slices() 함수는 train_images 배열을 받아서 데이터셋을 생성합니다. 이 데이터셋은 shuffle() 함수를 사용하여 버퍼 사이즈만큼의 샘플들을 섞고, batch() 함수를 사용하여 BATCH_SIZE만큼의 샘플들로 이루어진 배치(batch)로 묶어줍니다. 이 때, drop_remainder=True로 설정하면 데이터셋의 마지막 배치가 BATCH_SIZE보다 작은 경우 해당 배치를 버리게 됩니다. 마지막으로 prefetch() 함수를 사용하여 모델 학습 중 데이터를 CPU와 GPU 간에 비동기적으로 전달하며, 학습 효율을 향상시킵니다.

## 모델 만들기 

### 생성자

- 생성자는 시드값 (seed; 랜덤한 잡음)으로부터 CNN 의 역순으로 이미지를 생성하기 위해, `tf.keras.layers.Conv2DTranspose` (inverse Convolution 을 통한 upsampling) 층을 이용 합니다.
- stride=(2, 2) 일 경우 dimension 이 2 배로 upsampling 됩니다.
- 처음 `Dense`층은 시드값을 인풋으로 받습니다.
- 그 다음 원하는 사이즈 28x28x1의 이미지가 나오도록 Conv2DTranspose 를 이용한 업샘플링을 여러번 합니다.  
- tanh를 사용하는 마지막 층을 제외한 나머지 각 층마다 활성함수로 `tf.keras.layers.LeakyReLU`을 사용하고 있음을 주목하세요.

<img src="https://camo.githubusercontent.com/45e147fc9dfcf6a8e5df2c9b985078258b9974e3/68747470733a2f2f63646e2d696d616765732d312e6d656469756d2e636f6d2f6d61782f313030302f312a33394e6e6e695f6e685044614c7539416e544c6f57772e706e67" width="700">

- "same" padding and stride = 1, 출력은 같은 크기입니다  
- "same" padding and stride = 2, 출력은 두 배 크기입니다.

생성된 이미지를 보기 위해 우선 랜덤한 noise 벡터를 만들고 (아직 훈련이 되지않은) 생성자을 통해 이미지를 생성합니다.   
training=False인 경우 모델은 추론 모드이며, Dropout 및 BatchNormalization과 같은 레이어가 적용되지 않습니다.   
반면에 training=True인 경우 모델은 학습 모드이며, Dropout 및 BatchNormalization과 같은 레이어가 적용됩니다. 이는 학습 단계에서 가중치 업데이트를 위한 필요한 노이즈와 무작위성을 추가하기 위한 것입니다.

### 감별자 
- 감별자는 합성곱 신경망(Convolutional Neural Network, CNN) 기반의 이미지 분류기 입니다.  
- MNIST dataset 은 input_shape (28, 28, 1)  입니다.
- sigmoid output 은 probability scalar 값을 출력합니다.
- CNN 과의 차이 : pooling layer가 없고, stride 를 통하여 downsampling 을 합니다.

<img src="https://camo.githubusercontent.com/45e147fc9dfcf6a8e5df2c9b985078258b9974e3/68747470733a2f2f63646e2d696d616765732d312e6d656469756d2e636f6d2f6d61782f313030302f312a33394e6e6e695f6e685044614c7539416e544c6f57772e706e67" width="700">

- (아직까지 훈련이 되지 않은) 감별자를 사용하여, 생성된 이미지가 진짜인지 가짜인지 판별해 봅니다.  
- 모델은 진짜 이미지에는 positive, 가짜 이미지에는 negative 를 출력하도록 훈련될 것입니다.

## 손실함수와 옵티마이저 정의
- 두 모델의 손실함수와 옵티마이저를 별도로 정의 합니다.
- Discriminator는 output 이 sigmoid 이므로, binary crossentropy 를 loss function 으로 사용 합니다.
- . from_logits=True는 모델의 출력값이 logits 형태로 나오는 경우 사용됩니다. 





### 감별자 손실함수 

discriminator_loss 함수는 real_output과 fake_output 두 가지 인자를 받아, 진짜 이미지에 대한 감별자의 예측과 1로 이루어진 행렬을 비교하고, 가짜 (생성된) 이미지에 대한 감별자의 예측과 0으로 이루어진 행렬을 비교 합니다.

tf.ones_like(real_output)는 real_output과 같은 모양(shape)의 텐서를 생성하여 1로 채우고, tf.zeros_like(fake_output)는 fake_output과 같은 모양의 텐서를 생성하여 0으로 채웁니다. 이 두 텐서는 각각 진짜 데이터와 가짜 데이터에 대한 레이블을 나타냅니다.

real_loss는 real_output과 레이블이 1인 tf.ones_like(real_output) 간의 이진 교차 엔트로피 손실을 계산하고, fake_loss는 fake_output과 레이블이 0인 tf.zeros_like(fake_output) 간의 이진 교차 엔트로피 손실을 계산합니다.

- real image 는 label [11111..111], fake image 는 label [00000....000] 이 ground truth 가 됩니다.

마지막으로, total_loss는 real_loss와 fake_loss를 더한 값입니다. discriminator 는 real 은 real 로, fake 는 fake 로 바르게 판별해야 하므로 real_loss + fake_loss 가 minimize 되도록 training 합니다.

### 생성자 손실함수

- 감별자를 얼마나 잘 속였는지에 대해 수치화 합니다.
- generator_loss 함수는 fake_output 인자를 받아, 가짜 데이터에 대한 discriminator의 출력값을 반환 합니다. tf.ones_like(fake_output)는 fake_output과 같은 모양(shape)의 텐서를 생성하여 1로 채웁니다. 이 텐서는 가짜 데이터에 대한 레이블을 나타냅니다.
- 생성자가 원활히 수행되고 있다면, 감별자는 가짜 이미지를 진짜 (또는 1)로 분류를 할 것 입니다. 
- 생성된 이미지에 대한 감별자의 결정(fake_output)을 1로 이루어진 행렬과 비교합니다.(감별자가 감별한 결과가 모두 1 이 되어야 생성자가 감별자를 완벽히 속인 것임)  

### 감별자와 생성자는 따로 훈련되기 때문에, 감별자와 생성자의 옵티마이저는 별도 지정.

### 체크포인트 저장

- optimizer 와 model을 저장합니다. 
- 이렇게 생성된 체크포인트를 사용하면, 모델 학습 과정 중간에 중단되었을 경우 체크포인트 파일을 불러와서 학습을 이어나갈 수 있습니다.

## 훈련 루프 정의

- 훈련 루프는 생성자가 입력으로 랜덤시드를 받는 것으로부터 시작 합니다.
- 그 시드값을 사용하여 이미지를 생성 합니다.  
- 감별자를 사용하여 (훈련 세트에서 갖고온) 진짜 이미지와 (생성자가 생성해낸) 가짜이미지를 분류 합니다.
- 각 모델의 손실을 계산하고, 그래디언트 (gradients)를 사용해 생성자와 감별자를 업데이트 합니다. 
<img src="adverseModel.png" width="500">

## Training

<img src="GANtraining.png" width="500">

**이미지 생성 및 저장**

-`training`이 False로 맞춰진 것을 주목. 이렇게 하면 (배치정규화를 포함하여) 모든 층들이 추론 모드로 실행됩니다.

## 모델 훈련
- 위에 정의된 `train()` 메서드를 생성자와 감별자를 동시에 훈련하기 위해 호출  합니다.  
- 생성적 적대 신경망을 학습하는 것은 매우 까다롭다. 생성자와 감별자가 서로를 제압하지 않는 것이 중요 합니다. (비슷한 속도로 train) 
- 훈련 초반부에는 생성된 이미지는 랜덤 노이즈처럼 보이나 훈련이 진행될수록, 생성된 숫자는 점차 진짜와 비슷해집니다.  
- 약 50 에포크가 지난 후, MNIST 와 닮은 이미지가 생성 됩니다. 
- Colab에서 기본 설정으로 실행하면, 에포크마다 1분정도 소요 됩니다.

마지막 체크포인트를 복구합니다.

## GIF 생성